In [63]:
import pandas as pd
import numpy as np
books=pd.read_csv("books_classified.csv")

In [2]:
from transformers import pipeline

# Load model and tokenizer
model_name = "j-hartmann/emotion-english-distilroberta-base"
classifier=pipeline("text-classification", model=model_name,
                    top_k=None,
                    device=0)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

c:\New folder\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


In [66]:
classifier("i work for very low pay")

[[{'label': 'sadness', 'score': 0.7447106242179871},
  {'label': 'joy', 'score': 0.12758629024028778},
  {'label': 'anger', 'score': 0.10406024754047394},
  {'label': 'fear', 'score': 0.017243776470422745},
  {'label': 'surprise', 'score': 0.003209520597010851},
  {'label': 'neutral', 'score': 0.0017965221777558327},
  {'label': 'disgust', 'score': 0.0013930285349488258}]]

In [67]:
labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise', 'love','hope']


In [68]:
def classify_emotions(description,neutral_max=0.35):
    scores = {label: 0.0 for label in labels}
    count = 0

    for sentence in description.split("."):
        sentence = sentence.strip()
        if sentence:
            preds = classifier(sentence)[0]
            for pred in preds:
                scores[pred['label']] += pred['score']
            count += 1

    result = {}
    for label in scores:
        avg_score = scores[label] / count if count > 0 else 0.0
        if label == "neutral":
            avg_score = min(avg_score, neutral_max)  # cap neutral
        result[label] = round(avg_score, 3) #if avg_score > threshold else 0.0

    return result

In [69]:
emotion_df = books["description"].apply(classify_emotions).apply(pd.Series)
books = pd.concat([books, emotion_df], axis=1)


In [74]:
books.to_csv("books_with_emotions.csv", index=False)